In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn import preprocessing

In [2]:
train_file = '/home/ryan/cs/datasets/mercedes/train.csv'
test_file = '/home/ryan/cs/datasets/mercedes/test.csv'
train = pd.read_csv(train_file)
test = pd.read_csv(test_file)

In [3]:
for c in train.columns:
    if train[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train[c].values) + list(test[c].values)) 
        train[c] = lbl.transform(list(train[c].values))
        test[c] = lbl.transform(list(test[c].values))

In [7]:
usable_cols = train.columns.drop(['ID','y'])
grouped = train.groupby(list(usable_cols))

In [8]:
for name, group in grouped:
    if group.shape[0] > 1:
        print group.y

3512    121.57
3581    120.76
Name: y, dtype: float64
892     99.48
913    101.98
Name: y, dtype: float64
3287    105.08
3376     99.65
Name: y, dtype: float64
3070    106.96
3133    165.52
Name: y, dtype: float64
1735    106.52
1772    109.25
Name: y, dtype: float64
209    113.11
263    106.40
Name: y, dtype: float64
1547    110.47
1563    120.91
Name: y, dtype: float64
1560    125.91
1616    121.50
Name: y, dtype: float64
167    106.14
196    116.20
Name: y, dtype: float64
1689    110.88
1796    106.55
Name: y, dtype: float64
1981    103.85
2051    109.69
2063    105.94
Name: y, dtype: float64
3220    106.23
3325    116.93
Name: y, dtype: float64
3634    116.13
3706    104.57
3748    114.08
3818    116.36
Name: y, dtype: float64
3913    125.45
4009    108.69
Name: y, dtype: float64
1137    109.63
1176    122.65
Name: y, dtype: float64
2235    108.96
2325    108.05
Name: y, dtype: float64
2218    113.55
2305    121.79
Name: y, dtype: float64
2145    110.2
2149    108.8
Name: y, dtype:

In [9]:
X_trn = train.drop(['ID', 'y'], axis=1)
y_trn = train.y
X_test = test.drop('ID', axis=1)

In [10]:
print X_trn.shape
print X_test.shape

(4209, 376)
(4209, 376)


In [11]:
iso = IsolationForest(n_estimators=100, contamination=0.1, 
                     max_features=1.0, n_jobs=1, random_state=0,
                     verbose=0)


In [12]:
count = 0
mean_values = pd.DataFrame(columns=['ID', 'y'])
for name, group in grouped:
    if group.shape[0] > 1:
        iso.fit(group.drop('ID', axis=1))
        preds = iso.predict(group.drop('ID', axis=1))
        indices = []
        for i in range(len(preds)):
            if preds[i] == 1:
                indices.append(i)
        if len(indices) < 1: 
            indices = range(len(preds))
        mean = group.y.values[indices].mean()
        for i in group.ID.values:
            mean_values.loc[count] = [i, mean]
            count += 1

In [13]:
mean_values.ID = mean_values.ID.astype(int)

In [15]:
print 'Shape before merge: {}'.format(train.shape)
print 'Merging...'
merged_train = train.merge(mean_values, how='left', on='ID', suffixes=('','_mean'))
print 'Shape after merge: {}'.format(merged_train.shape)

Shape before merge: (4209, 378)
Merging...
Shape after merge: (4209, 380)


In [18]:
merged_train['y'] = np.where(np.isnan(merged_train['y_mean']), merged_train['y'], merged_train['y_mean'])

In [19]:
merged_train = merged_train.drop('y_mean', axis=1)

In [23]:
free_labels = test.merge(merged_train, how='left', on=list(usable_cols), suffixes=('','_extra'))

In [24]:
print free_labels.head()

   ID  X0  X1  X2  X3  X4  X5  X6  X8  X10       ...         X378  X379  X380  \
0   1  24  23  38   5   3  26   0  22    0       ...            1     0     0   
1   2  46   3   9   0   3   9   6  24    0       ...            0     0     0   
2   3  24  23  19   5   3   0   9   9    0       ...            1     0     0   
3   4  24  13  38   5   3  32  11  13    0       ...            1     0     0   
4   5  49  20  19   2   3  31   8  12    0       ...            0     0     0   

   X382  X383  X384  X385  ID_extra   y  duplicate_count  
0     0     0     0     0       NaN NaN              NaN  
1     0     0     0     0       NaN NaN              NaN  
2     0     0     0     0       NaN NaN              NaN  
3     0     0     0     0       NaN NaN              NaN  
4     0     0     0     0       NaN NaN              NaN  

[5 rows x 380 columns]
